In [ ]:
import numpy as np
import pandas as pd

Importing & Parsing the dataset as ratings and movies details

In [ ]:
ratingData = pd.io.parsers.read_csv('ratings.dat',names=['user_id', 'movie_id', 'rating', 'time'],engine='python', delimiter='::',error_bad_lines=False)
movieData = pd.io.parsers.read_csv('movies.dat',names=['movie_id', 'title', 'genre'],engine='python', delimiter='::',error_bad_lines=False,encoding='ISO-8859-1')
print(ratingData)


<ipython-input-2-6d10f30f6f52>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  ratingData = pd.io.parsers.read_csv('ratings.dat',names=['user_id', 'movie_id', 'rating', 'time'],engine='python', delimiter='::',error_bad_lines=False)


        user_id  movie_id  rating       time
0             1      1193       5  978300760
1             1       661       3  978302109
2             1       914       3  978301968
3             1      3408       4  978300275
4             1      2355       5  978824291
...         ...       ...     ...        ...
387843     2269      3408       4  981904540
387844     2269      3298       2  981904634
387845     2269      3481       5  974574930
387846     2269      2826       3  981904634
387847     2269      2827       1  981904634

[387848 rows x 4 columns]


<ipython-input-2-6d10f30f6f52>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  movieData = pd.io.parsers.read_csv('movies.dat',names=['movie_id', 'title', 'genre'],engine='python', delimiter='::',error_bad_lines=False,encoding='ISO-8859-1')


Create the ratings matrix of shape (m×u)

In [ ]:
ratingMatrix = np.ndarray(
    shape=(np.max(ratingData.movie_id.values), np.max(ratingData.user_id.values)),
    dtype=np.uint8)
ratingMatrix[ratingData.movie_id.values-1, ratingData.user_id.values-1] = ratingData.rating.values
print(ratingMatrix)

[[  5 179 145 ...   0   0   0]
 [  0   0   0 ...   0 137   2]
 [  0   0   0 ...   0   0   0]
 ...
 [ 61  41  58 ...  58   0   0]
 [  0   0 247 ... 244 153  40]
 [ 58   0   0 ...   0   0   0]]


Subtract Mean off - Normalization

In [ ]:
normalizedMatrix = ratingMatrix - np.asarray([(np.mean(ratingMatrix, 1))]).T
print(normalizedMatrix)

[[ -7.98148964 166.01851036 132.01851036 ... -12.98148964 -12.98148964
  -12.98148964]
 [-14.74878801 -14.74878801 -14.74878801 ... -14.74878801 122.25121199
  -12.74878801]
 [-14.96253856 -14.96253856 -14.96253856 ... -14.96253856 -14.96253856
  -14.96253856]
 ...
 [ 23.5570736    3.5570736   20.5570736  ...  20.5570736  -37.4429264
  -37.4429264 ]
 [-46.0405465  -46.0405465  200.9594535  ... 197.9594535  106.9594535
   -6.0405465 ]
 [ 16.37990304 -41.62009696 -41.62009696 ... -41.62009696 -41.62009696
  -41.62009696]]


Computing SVD

In [ ]:
A = normalizedMatrix.T / np.sqrt(ratingMatrix.shape[0] - 1)
U, S, V = np.linalg.svd(A)

Calculate cosine similarity, sort by most similar and return the top N

In [ ]:
def similar(ratingData, movie_id, top_n):
    index = movie_id - 1 # Movie id starts from 1
    movie_row = ratingData[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', ratingData, ratingData)) #Einstein summation |  traditional matrix multiplication and is equivalent to np.matmul(a,b)
    similarity = np.dot(movie_row, ratingData.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity) #Perform an indirect sort along the given axis (Last axis)
    return sort_indexes[:top_n]


Select k principal components to represent the movies, a movie_id to find recommendations and print the top_n results

In [ ]:
k = 50
movie_id = 2
top_n = 5

sliced = V.T[:, :k] # representative data
indexes = similar(sliced, movie_id, top_n)

print('Recommendations for Movie {0}: \n'.format(
movieData[movieData.movie_id == movie_id].title.values[0]))
for id in indexes + 1:
    print(movieData[movieData.movie_id == id].title.values[0])

Recommendations for Movie Jumanji (1995): 

Jumanji (1995)
That Thing You Do! (1996)
Bambi (1942)
Superman III (1983)
Police Academy (1984)
